# Implementing a CNN for CIFAR-10 classification:

## using PyTorch


## import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

##  Define the CNN architecture

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

##  Load and preprocess the CIFAR-10 dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:01<00:00, 103805271.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


##  Initialize the model, loss function, and optimizer

In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Train the model

In [5]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

Epoch 1, Loss: 1.2915401914540459
Epoch 2, Loss: 0.8905380308018316
Epoch 3, Loss: 0.6985640084880698
Epoch 4, Loss: 0.5328258204909847
Epoch 5, Loss: 0.37390131182263575
Epoch 6, Loss: 0.2392826494486893
Epoch 7, Loss: 0.14687354332241026
Epoch 8, Loss: 0.09450330904415806
Epoch 9, Loss: 0.07736492001325311
Epoch 10, Loss: 0.06645500760816057


## Evaluate the model

In [6]:


correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")

Accuracy on test set: 72.27%


## Potential improvements:

1.   Use data augmentation techniques
2.   Implement learning rate scheduling
3.   Try different architectures (e.g., ResNet, VGG)
4.   Use regularization techniques (e.g., dropout, weight decay)








# 2.Feedforward neural network for housing price prediction:

## import the libraries

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load and preprocess the data

In [9]:
data = pd.read_csv('housing_prices.csv')
X = data.drop('Price', axis=1)
y = data['Price']


## Encode categorical variables


In [10]:
X = pd.get_dummies(X, columns=['Location'])

## Split the data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Normalize the input features

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Convert to PyTorch tensors

In [13]:
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

## Define the neural network

In [14]:
class HousingNN(nn.Module):
    def __init__(self, input_size):
        super(HousingNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Initialize the model, loss function, and optimizer

In [15]:
input_size = X_train_scaled.shape[1]
model = HousingNN(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train the model

In [16]:
num_epochs = 1000
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 138072817664.0000
Epoch [200/1000], Loss: 137963257856.0000
Epoch [300/1000], Loss: 137629859840.0000
Epoch [400/1000], Loss: 136950161408.0000
Epoch [500/1000], Loss: 135818543104.0000
Epoch [600/1000], Loss: 134151208960.0000
Epoch [700/1000], Loss: 131884064768.0000
Epoch [800/1000], Loss: 128892387328.0000
Epoch [900/1000], Loss: 125198508032.0000
Epoch [1000/1000], Loss: 120816345088.0000


## Evaluate the model

In [17]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    mse = criterion(y_pred, y_test_tensor)
    print(f'Mean Squared Error on test set: {mse.item():.4f}')

Mean Squared Error on test set: 85973598208.0000
